In [1]:
import numpy as np

from pathlib import Path
pathtohere = Path.cwd()
print(pathtohere)

C:\Users\02dba\Documents\GitHub\Diatomic-Molecule-Dissociation-by-Gravitational-Gradient\contents


In [2]:
def formatExpressions(motionEquations):
    """
    
    
    """
    
    patterns = {
        'q' : 'p->charge',
        'sigma(r,theta)': 'sigma_p',
        'delta(r)': 'delta_p',
        'chi(theta)': 'chi_p',
        'Power(Q,2)': 'BH->charge2',
        'Power(l,2)': 'BH->l2',
        'Q': 'BH->charge',
        'l': 'BH->l',
        'a': 'BH->a',
        'M': 'BH->mass',
        'Cos': 'cos',
        'Sin': 'sin',
        'Csc': '1. / sin',
        'Cot': '1. / tan',
        'Sec': '1. / cos',
        'm' : 'p->mass',
        'r': 'p->r',
        'phi': 'p->phi',
        'theta': 'p->theta',
        'Power(q,2)': 'p->charge*ps->charge',
        'angular' : 'p->L',
        'energy' : 'p->energy',
        'Power(r,2)' : 'p->r2',
        'rs': 'ps->r',
        'phis': 'ps->phi',
        'thetas': 'ps->theta',
        'Power(r,2)' : 'p->r2',
        # Keep the same.
        '1.*' : '',
        '2.' : '2.',
        '/2' : '/2.',
        'Power' : 'pow',
        'epsilon0' : 'epsilon_0',
        'nmag' : 'nmag',
        'tdot': 'tdot',
        'rdot' : 'rdot',
        'phidot' : 'phidot',
        'thetadot' : 'thetadot',
        'tsdot': 'tsdot',
        'rsdot' : 'rsdot',
        'phisdot' : 'phisdot',
        'thetasdot' : 'thetasdot',
        'tsddot':'tsddot',
        'rsddot' : 'rsddot',
        'phisddot':'phisddot',
        'thetasddot' : 'thetasddot'
    }
    
    maxPatternLength = max(map(len, patterns.keys()))
    
    for i in range(len(motionEquations)):
        j = 0
        while j < len(motionEquations[i]):
            for k in range(0,maxPatternLength):
                section = motionEquations[i][j:j+maxPatternLength - k] 
                #print(section)
                
                if section in patterns.keys():
                    motionEquations[i] = (motionEquations[i][:j] + patterns[section] 
                                        + motionEquations[i][j+maxPatternLength - k:])
                    j += len(patterns[section]) - 1
                    break
                    
            j += 1
                
                
                    
    return motionEquations



def configureFile():
    """
    Configure the cpp file to find the new coordinates from the Euler-Lagrange equations.
    """
    
    configureIdentifier = '// Configured==true\n'
    
    motionEquations = []
    
    line = True
    filePath = pathtohere / 'src/eulerLagrange.cpp'
    i = 0

    # https://stackoverflow.com/questions/3277503/how-to-read-a-file-line-by-line-into-a-list
    with open(filePath, 'r', encoding='UTF-8') as f:
        while line and i < 4:
            line = f.readline()
            
            if line == configureIdentifier:
                print(f'File already configured.')
                return
            if i==0:
                print(f'File not configured: {configureIdentifier[:-1]} not found.')
            
            
            cut = slice(0,-1) if i<3 else slice(0,None)
            motionEquations.append(line[cut])
            
            i+=1
            
    motionEquations = formatExpressions(motionEquations)
    
    with open(filePath, 'w', encoding='UTF-8') as f:
        f.write(f'{configureIdentifier}')
        f.write("#include \"eulerLagrange.h\"\n\n")

        f.write('const double epsilon_0{ 1. / (4.*M_PI) }; // Geometrised-Gaussian units\n\n\n')
        
        f.write('std::tuple<double, double, double, double> eulerMoveMathematica(const BlackHole* BH,\n')
        f.write('\tParticle* p, Particle* ps,\n')
        f.write('\tconst double sigma_p, const double delta_p, const double chi_p, const double dlambda)\n')
        f.write('{\n')
        f.write('\tdouble t_new{};\n')
        f.write('\tdouble r_new{};\n')
        f.write('\tdouble phi_new{};\n')
        f.write('\tdouble theta_new{};\n\n')
        
        f.write('\tconst double tdot{ (p->t - p->t_prev) / dlambda };\n')
        f.write('\tconst double rdot{ (p->r - p->r_prev) / dlambda };\n')
        f.write('\tconst double phidot{ (p->phi - p->phi_prev) / dlambda };\n')
        f.write('\tconst double thetadot{ (p->theta - p->theta_prev) / dlambda };\n\n')
        f.write('\tconst double tsdot{ (ps->t - ps->t_prev) / dlambda };\n')
        f.write('\tconst double rsdot{ (ps->r - ps->r_prev) / dlambda };\n')
        f.write('\tconst double phisdot{ (ps->phi - ps->phi_prev) / dlambda };\n')
        f.write('\tconst double thetasdot{ (ps->theta - ps->theta_prev) / dlambda };\n')
        f.write('\tconst double tsddot{(ps->t - 2.*ps->t_prev + ps->t_prevprev) / (dlambda*dlambda)};\n')
        f.write('\tconst double rsddot{(ps->r - 2.*ps->r_prev + ps->r_prevprev) / (dlambda*dlambda)};\n')
        f.write('\tconst double phisddot{(ps->phi - 2.*ps->phi_prev + ps->phi_prevprev) / (dlambda*dlambda)};\n')
        f.write('\tconst double thetasddot{(ps->theta - 2.*ps->theta_prev + ps->theta_prevprev) / (dlambda*dlambda)};\n')
        
        f.write('\tconst double nx{p->r*sin(p->theta)*cos(p->phi) - ps->r*sin(ps->theta)*cos(ps->phi)};\n') 
        f.write('\tconst double ny{p->r*sin(p->theta)*sin(p->phi) - ps->r*sin(ps->theta)*sin(ps->phi)};\n') 
        f.write('\tconst double nz{p->r*cos(p->theta) - ps->r*cos(ps->theta)};\n')
        f.write('\tconst double nmag{sqrt(nx*nx + ny*ny + nz*nz)};\n\n')
        
        f.write(f'\tt_new = 2. * p->t - p->t_prev + dlambda*dlambda*({motionEquations[0]});\n')
        f.write(f'\tr_new = 2. * p->r - p->r_prev + dlambda*dlambda*({motionEquations[1]});\n')
        f.write(f'\tphi_new = 2. * p->phi - p->phi_prev + dlambda*dlambda*({motionEquations[2]});\n')
        f.write(f'\ttheta_new = 2. * p->theta - p->theta_prev + dlambda*dlambda*({motionEquations[3]});\n\n')
        
        f.write('\treturn {t_new, r_new, phi_new, theta_new};\n')
        f.write('}\n')
        
        
    filePath = pathtohere / 'include/eulerLagrange.h'
    with open(filePath, 'w', encoding='UTF-8') as f:
        f.write('#ifndef EULERLAGRANGE_H\n')
        f.write('#define EULERLAGRANGE_H\n\n')
        
        f.write('#include <iostream>\n')
        f.write('#include <tuple>\n')
        f.write('#define _USE_MATH_DEFINES\n')
        f.write('#include "math.h"\n\n')


        f.write('#include "blackHole.h"\n')
        f.write('#include "particle.h"\n\n\n')
        f.write('std::tuple<double, double, double, double> eulerMoveMathematica(const BlackHole* BH,\n')
        f.write('\tParticle* p, Particle* ps,\n')
        f.write('\tconst double sigma_p, const double delta_p, const double chi_p, const double dlambda);\n\n')

        f.write('#endif EULERLAGRANGE_H\n')
            
                        
            

In [3]:
def main():
    configureFile()

In [4]:
if __name__=='__main__':
    main()

File not configured: // Configured==true not found.
